In [1]:
using CSV, DataFramesMeta, Query, PlotlyJS, FloatingTableView, Dates

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10878882322143915528\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13967461402872197770\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10878882322143915528\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
df = CSV.File("spec20210723.tsv") |> DataFrame;

In [3]:
# FloatingTableView package의 browse 함수를 이용하면 dataframe을 별도의 창에 띄울 수 있다.
# browse(df)

In [4]:
# PRODUCT_CODE를 string type으로 바꾸고 PLANT 컬럼과 합쳐서 Plantcode 컬럼에 저장하기
@transform! df begin    
    :Plantcode = :PLANT .* string.(:PRODUCT_CODE)
end;

In [5]:
# EFFECTIVITY_DATE 컬럼을 나누어서 date에 해당하는 값만 취한 후 날짜 타입 데이터로 변경하기
# missing value 처리를 할 수 없어서 broadcasting을 못하고 for loop를 사용함
# df[!, :e_date] = 
#   [ismissing(df[idx, :EFFECTIVITY_DATE]) ? 
#     missing : 
#     Date(split(df[idx, :EFFECTIVITY_DATE], " ")[1],"mm/dd/yyyy") 
#     for idx in 1:size(df)[1]];

In [6]:
# 위 코드에 대한 vectorization expression.  passmissing 함수를 찾아내어 성공함
# 문자열 EFFECTIVITY_DATE 컬럼을 split 함수를 이용하여 공백을 기준으로 나누고
# 나누어진 첫번째 컬럼(날짜에 대한 문자열)을 date 데이터 타입으로 변경한다.
@transform! df @byrow @passmissing :e_date = Date(split(:EFFECTIVITY_DATE, ' ')[1], "mm/dd/yyyy");

In [7]:
sort!(df, [:Plantcode, :e_date]);

In [8]:
# grouped dataframe의 맨 마지막 row를 뽑아내는 표현
# 각 제품의 최신 제조 사양만 남는다.
gdf = groupby(df, :Plantcode)
dfl = combine(last, gdf);

# 현재 운영 중인 상품만 추출
@subset!(dfl, :SPEC_STATE .== "Release", :PRODUCT_STATE .== "Active");

# 다원화 관계의 제품 선별 규칙

다음 컬럼들의 값이 동등하다면 다원화 관계로 묶을 수 있다.
- SIZE_FULL, PATTERN, PLY_RATING, BW_INDICATOR
- PRE_BELT_TYPE, PLYS(원래 PLY_LOCK이 같아야 하지만 성형 방식에 따라 표현이 달라질 수 있으므로 PLYS로 환산하여 사용)
- MOLD_OD, MOLD_SH, MOLD_SW, MOLD_SD, MOLD_RW, MOLD_TDW
- MOLD_TR1, MOLD_TR2, RPB_TYPE
- CTB_COMPOUND, SUT_COMPOUND
> PLYS는 PLY_LOCK의 값을 "-"를 기준으로 나눈 다음 숫자로 바꾸고 더하여 구한다.

Studding 여부, foam의 부착 여부에 대한 구분을 TL_INDICATOR에 표시하기 시작하였다.  
다원화 관계를 정의할 때, 이를 어떻게 처리할 지 결정이 필요하다.  

다음 컬럼들은 위의 기준에 포함시킬 지 여부를 고민해야 한다.
- OVER
- C01_ROLLED, C02_ROLLED, BT1_ROLLED, BT2_ROLLED
- C01_MATERIAL, C02_MATERIAL, C03_MATERIAL, BT1_MATERIAL
- PRE_BT1_PART_CODE3, PRE_JLB_JLC_PART_CODE3
- MOLD_CODE

ORIGINAL_CUSTOMER 컬럼은 값이 RE일 때만 다원화 관계가 가능하다.

In [9]:
# PLYS 컬럼
@transform! dfl @byrow :PLYS = parse(Int, split(:PLY_LOCK, '-')[1]) + parse(Int, split(:PLY_LOCK, '-')[2]);

In [10]:
# 보강구조를 나타내는 PRE_BELT_TYPE에 missing값이 있음.
# 이를 채우기 위해 사용할 컬럼
nf_cols = ["NEC_LENGTH", "NFC_LENGTH"];
jf_cols = ["JEC_WIDTH", "JEC_LENGTH", "JFC_WIDTH", "JFC_LENGTH"];
jl_cols = ["JLC_TYPE"];

In [11]:
# 사용할 컬럼을 정의
id_cols = [
    :Plantcode, :PLANT, :USE_CODE, :BRAND
];

critical_cols = [
    :SIZE_FULL, :PATTERN, :PLY_RATING, :BW_INDICATOR,
    :PLYS, :PRE_BELT_TYPE,
    :MOLD_OD, :MOLD_SH, :MOLD_SW, :MOLD_SD, :MOLD_RW, :MOLD_TDW,
    :MOLD_TR1, :MOLD_TR2, :RPB_TYPE,
    :CTB_COMPOUND, :SUT_COMPOUND,
    :ORIGINAL_CUSTOMER
];

reference_cols = [
    :OVER,
    :C01_ROLLED, :C02_ROLLED, :BT1_ROLLED, :BT2_ROLLED,
    :C01_MATERIAL, :C02_MATERIAL, :C03_MATERIAL, :BT1_MATERIAL,
    :PRE_BT1_PART_CODE3, :PRE_JLB_JLC_PART_CODE3,
    :MOLD_CODE,
    :BELT_DRUM_CIR
];

# 
dr = select(
    dfl,
    id_cols..., critical_cols..., reference_cols..., nf_cols..., jf_cols..., jl_cols...
    );

sort!(dr, [:SIZE_FULL, :PATTERN]);

In [12]:
# size와 pattern이 같은 경우에 대해 고유번호를 정하고 id_1st 컬럼에 저장한다.
# id_1st 값이 같은 row는 size와 pattern이 같은 제품들임.
size_pattern_list = unique(select(dr, [:SIZE_FULL, :PATTERN]), [:SIZE_FULL, :PATTERN]);
@transform!(size_pattern_list, :id_1st = 1:nrow(size_pattern_list));

# dr 데이터프레임에 id_1st 배치
dr = leftjoin(dr, size_pattern_list, on=[:SIZE_FULL, :PATTERN]);

In [13]:
# ORIGINAL_CUSTOMER가 "OE"인 제품 제외.
# 이들은 다원화 상품이 될 수 없음.
@subset!(dr, :ORIGINAL_CUSTOMER .!= "OE");

In [14]:
# 중복이 존재하는 id_1st의 값 추출
duplicated_id_1st = unique(dr[findall(nonunique(dr, :id_1st)), :id_1st]);

# 중복이 존재하는 제품 리스트
# 이 리스트에서 다원화 제품을 검색한다.
dr = dr[findall(in(duplicated_id_1st), dr.id_1st), :];

In [15]:
# NEC와 NFC 적용품이 있는지 조사
nrow(dr) - sum(ismissing.(dr.NEC_LENGTH)),
nrow(dr) - sum(ismissing.(dr.NFC_LENGTH))

(0, 0)

In [16]:
# NEC, NFC 적용품은 없으므로 해당 컬럼 삭제
select!(dr, Not(r"NEC|NFC"));

In [17]:
# JEC와 JFC 적용품이 있는지 조사
dummy = dr[ismissing.(dr.PRE_BELT_TYPE), :]
nrow(dummy) - sum(ismissing.(dummy.JEC_LENGTH)),
nrow(dummy) - sum(ismissing.(dummy.JFC_LENGTH))

(2, 2)

In [18]:
dr[
    (dr.JFC_LENGTH .!== missing) .& ismissing.(dr.PRE_BELT_TYPE),
    vcat(id_cols, [:SIZE_FULL, :JFC_LENGTH, :JEC_LENGTH, :BELT_DRUM_CIR])
]

,Plantcode,PLANT,USE_CODE,BRAND,SIZE_FULL,JFC_LENGTH,JEC_LENGTH,BELT_DRUM_CIR
,String,String,String,String,String,Int64?,Int64?,Float64?
1,JP1007410,JP,C-,HK,265/70R17S,52866,13040,2332.0
2,JP1010789,JP,CA,AU,265/70R17S,52866,13040,2332.0


In [19]:
# PRE_BELT_TYPE이 missing이고, JFC_LENGTH와 JEC_LENGTH가 missing이 아닌 row는 2개이다.
# 2개 모두 보강 구조는 JF757로 추정된다.
dr[
    (dr.JFC_LENGTH .!== missing) .& ismissing.(dr.PRE_BELT_TYPE),
    :PRE_BELT_TYPE
] .= "JF757";

In [20]:
# 남은 missing들은 None으로 바꾼다. 보강구조가 없다는 의미이다.
dr.PRE_BELT_TYPE = replace(dr.PRE_BELT_TYPE, missing => "None");

In [21]:
# PRE_BELT_TYPE의 missing value 처리가 완료되었으므로 필요없는 컬럼을 제거한다.
dr = select(dr, Not(vcat(jf_cols, jl_cols)));
dr = select(dr, Not(:BELT_DRUM_CIR));

In [22]:
# size, pattern, ply rating, black or white, carcass number, 보강구조가 같은 경우에 대해 고유번호를 정하고 id_str 컬럼에 저장한다.
# id_2nd 값이 같은 row는 위의 컬럼이 같은 제품들임.
cols = [:SIZE_FULL, :PATTERN, :PLY_RATING, :BW_INDICATOR, :PRE_BELT_TYPE, :PLYS]

list_struct = unique(
    select(dr, cols), cols
);
@transform!(list_struct, :id_struct = 1:nrow(list_struct));

# dr 데이터프레임에 id_struct 배치
dr = leftjoin(dr, list_struct, on=cols);

In [23]:
# MOLD_OD, MOLD_SD, MOLD_SW에 missing value가 얼마나 있는지 조사. 없는 것으로 나타남.
sum(ismissing.(dr.MOLD_OD)), sum(ismissing.(dr.MOLD_SD)), sum(ismissing.(dr.MOLD_SW))

(0, 0, 0)

In [24]:
# 위에 더해 MOLD_OD, MOLD_SW, MOLD_SD까지 같은 경우에 대해 고유번호를 정하고 id_mold 컬럼에 저장한다.
# id_mold 값이 같은 row는 위의 컬럼이 같은 제품들임.
cols = [
    :SIZE_FULL, :PATTERN, :PLY_RATING, :BW_INDICATOR,
    :PRE_BELT_TYPE, :PLYS,
    :MOLD_OD, :MOLD_SW, :MOLD_SD
]

list_mold = unique(
    select(dr, cols), cols
);
@transform!(list_mold, :id_mold = 1:nrow(list_mold));

# dr 데이터프레임에 id_mold 배치
dr = leftjoin(dr, list_mold, on=cols);

In [25]:
# CTB_COMPOUND, SUT_COMPOUND에 missing value가 얼마나 있는지 조사. 없는 것으로 나타남.
sum(ismissing.(dr.CTB_COMPOUND)), sum(ismissing.(dr.SUT_COMPOUND))

(0, 2)

In [26]:
dr[
    ismissing.(dr.SUT_COMPOUND),
    vcat(id_cols, [:SIZE_FULL, :PATTERN])
]

,Plantcode,PLANT,USE_CODE,BRAND,SIZE_FULL,PATTERN
,String,String,String,String,String,String
1,DP1019216,DP,N,HL,160/650R15,SR10W
2,DP1019219,DP,N,HR,160/650R15,SR10W


In [27]:
# missing들은 None으로 바꾼다. Subtread compound가 없는 제품으로 판단된다.
dr.SUT_COMPOUND = replace(dr.SUT_COMPOUND, missing => "None");

In [28]:
# 위에 더해 CTB_COMPOUND, SUT_COMPOUND까지 같은 경우에 대해 고유번호를 정하고 id_compound 컬럼에 저장한다.
# id_compound 값이 같은 row는 위의 컬럼이 같은 제품들임.
cols = [
    :SIZE_FULL, :PATTERN, :PLY_RATING, :BW_INDICATOR,
    :PRE_BELT_TYPE, :PLYS,
    :MOLD_OD, :MOLD_SW, :MOLD_SD,
    :CTB_COMPOUND, :SUT_COMPOUND
]

list_compound = unique(
    select(dr, cols), cols
);
@transform!(list_compound, :id_compound = 1:nrow(list_compound));

# dr 데이터프레임에 id_mold 배치
dr = leftjoin(dr, list_compound, on=cols);

In [29]:
browse(dr)